In [67]:
pip install pyspark

In [68]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Conservação do acervo") \
    .getOrCreate()


In [69]:
df = spark.read.csv("/content/Dados abertos 2022_CPP_COP_Conservação preventiva do acervo.csv", header=True, inferSchema=True, sep=';', encoding='latin1')

In [70]:
# Mostrando as primeiras linhas do DataFrame
df.show()

# Imprimindo o esquema do DataFrame para ver as colunas e tipos de dados
df.printSchema()

# Contando o número de linhas
print(f"Número de linhas: {df.count()}")

+--------------------+-----------+-----------------------+
|         Localização|      Setor|Mês e ano procedimento |
+--------------------+-----------+-----------------------+
|Livros Raros  CII...|Obras Raras|                03 2022|
|Livros Raros  CII...|Obras Raras|                03 2022|
|Livros Raros  CII...|Obras Raras|                03 2022|
|Livros Raros  CII...|Obras Raras|                03 2022|
|Livros Raros  CII...|Obras Raras|                03 2022|
|Livros Raros Z002...|Obras Raras|                03 2022|
|Livros Raros Z002...|Obras Raras|                03 2022|
|        Z001,002,004|Obras Raras|                03 2022|
|        Z001,003,010|Obras Raras|                03 2022|
|        Z001,003,016|Obras Raras|                03 2022|
|     Z001,003BIS,002|Obras Raras|                03 2022|
|Livros Raros  Z00...|Obras Raras|                03 2022|
|        Z001,002,013|Obras Raras|                04 2022|
|        Z001,001,012|Obras Raras|                04 202

In [71]:
# Removendo linhas com valores nulos
df = df.na.drop()

# Removendo linhas duplicadas
df = df.dropDuplicates()

In [72]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import calendar
import re

# Atualizando a função UDF para padronizar as datas
def padronizar_data(data_str):
    # Tentativa de padronizar a partir do formato 'MM YYYY'
    match = re.match(r'(\d{2})\s(\d{4})', data_str)
    if match:
        return '-'.join(match.groups())

    # Tentativa de padronizar a partir do formato 'mmm/yy' ou 'mmm/yyyy'
    match = re.match(r'([a-zA-Z]+)\/(\d{2,4})', data_str)
    if match:
        mes_str, ano = match.groups()
        # Convertendo o nome do mês para número
        try:
            mes = list(calendar.month_abbr).index(mes_str.capitalize())
        except ValueError:
            mes = 0  # Mes desconhecido
        # Ajustando o ano para quatro dígitos, se necessário
        ano = '20' + ano if len(ano) == 2 else ano
        return f'{mes:02d}-{ano}'

    # Se nenhum formato conhecido foi encontrado, retorna a string original
    return data_str

# Criando UDF com a função de padronização de datas
padronizar_data_udf = udf(padronizar_data, StringType())

# Nome da coluna corrigido
nome_coluna_data = "Mês e ano procedimento "

# Aplicando a UDF à coluna de data
df = df.withColumn(nome_coluna_data, padronizar_data_udf(col(nome_coluna_data)))

# Mostrando os resultados para verificar a conversão
df.select(nome_coluna_data).show()

+-----------------------+
|Mês e ano procedimento |
+-----------------------+
|                10-2022|
|                04-2022|
|                10-2022|
|                11-2022|
|                06-2022|
|                07-2022|
|                03-2022|
|                09-2022|
|                04-2022|
|                07-2022|
|                07-2022|
|                06-2022|
|                10-2022|
|                12-2022|
|                06-2022|
|                07-2022|
|                10-2022|
|                07-2022|
|                11-2022|
|                11-2022|
+-----------------------+
only showing top 20 rows



In [73]:
df.show()

+------------------+-----------+-----------------------+
|       Localização|      Setor|Mês e ano procedimento |
+------------------+-----------+-----------------------+
|      Z002,001,002|Obras Raras|                10-2022|
|      Z002,001,005|Obras Raras|                04-2022|
|      Z001,004,002|Obras Raras|                10-2022|
|      Z001,001,009|Obras Raras|                11-2022|
|   Z002,002BIS,003|Obras Raras|                06-2022|
|      Z001,003,017|Obras Raras|                07-2022|
|   Z001,003BIS,002|Obras Raras|                03-2022|
|      Z001,003,019|Obras Raras|                09-2022|
|      Z002,002,012|Obras Raras|                04-2022|
|      Z002,001,019|Obras Raras|                07-2022|
|      Z002,002,018|Obras Raras|                07-2022|
|      Z002,003,008|Obras Raras|                06-2022|
|Z002,004,004 n.002|Obras Raras|                10-2022|
|      Z001,003,001|Obras Raras|                12-2022|
|      Z002,004,019|Obras Raras

In [74]:
df_agrupado = df.groupBy('Mês e ano procedimento ').count().orderBy('Mês e ano procedimento ')
df_agrupado.show()

+-----------------------+-----+
|Mês e ano procedimento |count|
+-----------------------+-----+
|                00-2022|    1|
|                03-2022|   12|
|                04-2022|   19|
|                06-2022|   17|
|                07-2022|   32|
|                09-2022|   16|
|                10-2022|   18|
|                11-2022|   18|
|                12-2022|   12|
+-----------------------+-----+



In [75]:
df.describe()

DataFrame[summary: string, Localização: string, Setor: string, Mês e ano procedimento : string]

In [76]:
df.write.csv("original/para/dataset_processado.csv", header=True)

In [77]:
spark.stop()